In [1147]:
import numpy as np
import pandas as pd 
import xgboost as xgb
import glob 
import re
from collections import defaultdict
from itertools import compress
import string 
from string import punctuation
import nltk
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
st = PorterStemmer()
stop = stopwords.words('english')
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import Word
from gensim import corpora
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.test.utils import common_texts
from pprint import pprint 
from random import shuffle
from scipy import spatial
from scipy.spatial.distance import cdist
from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.filterwarnings('ignore')

In [1148]:
def clean_string(string,split = True):
    string = re.sub(r'\.\.+', '', string).replace('.', '')         ## Add space after '.'
    string = re.sub('\d+(?:\.\d+)?', '', string)                   ## remove numbers
           
    if split == True:
        string = string.split('\n\n-----\n\n')                     ## spilt paragraphs
        string =[re.sub(r'\([^\(]*?\)', r'', x) for x in string]   ## remove {},() and content between them 
        string = [re.sub('[^A-Za-z0-9]+', ' ', x) for x in string]     
    else:
        string =re.sub(r'\([^\(]*?\)', r'', string)
        string = re.sub('[^A-Za-z0-9]+', ' ', string)              ## remove other punctuations
        
    return string

In [1149]:
## spilts the txt file into sentences considering almost all edges cases

caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co|jr)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"  
single_dot_acronyms = r'(?:[A-Z]\.)+'
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)    ## Do not spilit acronyms of form A.B.C
    text = re.sub(caps + "[.]" + caps + caps,"\\1<prd>\\2<prd>",text)     ## Do not spilit acronyms of form A.BC
    text = re.sub(caps + "[.]" + caps,"\\1<prd>\\2<prd>",text)              
    text = re.sub(" "+"[.]" + caps+caps,"\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)   
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [1150]:
## case low and remove stop words from documents

stoplist = stopwords.words('english')
def remove_stopwords(docs):
    return [[word for word in doc.lower().split() if word not in stoplist] for doc in docs]

In [1151]:
## remove words that just occurs once

def remove_less_frequent(texts):
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    return [[token for token in text if frequency[token] > 1]for text in texts]


In [1152]:
## cosine similarity of two 1D vectors
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [1153]:
## each topic (paragraph) in a txt file as a document

documents=[]
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/training-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    data = clean_string(data)
    documents=documents + data
    
## each text file as a document

documents_whole=[]
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/training-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    data = clean_string(data,False)
    documents_whole = documents_whole + [data]
    
## each pair of sequential sentences as a document

documents_sentences_pairs=[]
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/training-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    data = clean_string(data)
    documents_sentences_pairs = documents_sentences_pairs+ data

## pair the sentences
documents_sentences_pairs = [i+j for i,j in zip(documents_sentences_pairs[::2], documents_sentences_pairs[1::2])]

In [1154]:
documents[0]

'I think that should be welcomed The Fed of course is taking this action to strengthen growth in America and I think we can all benefit from that over time On global currency movements We don t want to see competitive devaluations occurring in the system It s very important that we do over time deal with these global imbalances and of course some of the structural problems in the system are wider than just exchange rates '

In [1155]:
documents_whole[0]

'I think that should be welcomed The Fed of course is taking this action to strengthen growth in America and I think we can all benefit from that over time On global currency movements We don t want to see competitive devaluations occurring in the system It s very important that we do over time deal with these global imbalances and of course some of the structural problems in the system are wider than just exchange rates Leaders of the member APEC bloc will meet on November in Yokohama neat Tokyo while G countries will also hold a summit next week in neighboring South Korea Following are key quotes from the APEC meeting CHINESE VICE FINANCE MINISTER WANG JUN Nov news conference The quantitative easing monetary policies adopted by the United States will boost the US economy and boosting the US economy will play an important role in global economic recovery At the same time the quantitative easing policy has already prompted the concern of emerging nations and we will continue paying att

In [1156]:
documents_sentences_pairs[0]

'I think that should be welcomed The Fed of course is taking this action to strengthen growth in America and I think we can all benefit from that over time On global currency movements We don t want to see competitive devaluations occurring in the system It s very important that we do over time deal with these global imbalances and of course some of the structural problems in the system are wider than just exchange rates Leaders of the member APEC bloc will meet on November in Yokohama neat Tokyo while G countries will also hold a summit next week in neighboring South Korea Following are key quotes from the APEC meeting CHINESE VICE FINANCE MINISTER WANG JUN Nov news conference The quantitative easing monetary policies adopted by the United States will boost the US economy and boosting the US economy will play an important role in global economic recovery'

In [1157]:
## including val data too

In [1158]:
## each topic (paragraph) in a txt file as a document 

documents_val=[]
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/val-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    data = clean_string(data)
    documents_val = documents_val + data
    
## each text file as a document

documents_val_whole=[]
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/val-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    data = clean_string(data,False)
    documents_val_whole = documents_val_whole + [data]
    
## each pair of sequential sentences as a document

documents_val_sentences_pairs=[]
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/val-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    data = clean_string(data)
    documents_val_sentences_pairs = documents_val_sentences_pairs + data

## pair the sentences
documents_val_sentences_pairs = [i+j for i,j in zip(documents_val_sentences_pairs[::2], documents_val_sentences_pairs[1::2])]

In [1159]:
texts = remove_stopwords(documents)
texts_whole = remove_stopwords(documents_whole)
texts_pairs = remove_stopwords(documents_sentences_pairs)

texts_val = remove_stopwords(documents_val)
texts_whole_val = remove_stopwords(documents_val_whole)
texts_pairs_val = remove_stopwords(documents_val_sentences_pairs)

In [1163]:
# x=texts[:]
# y=texts_whole_val[:]

In [1164]:
texts = remove_less_frequent(texts)
texts_whole = remove_less_frequent(texts_whole)
texts_pairs = remove_less_frequent(texts_pairs)

texts_val = remove_less_frequent(texts_val)
texts_whole_val = remove_less_frequent(texts_whole_val)
texts_pairs_val = remove_less_frequent(texts_pairs_val)

In [1166]:
print len(texts),len(texts_whole),len(texts_pairs),

11051 2234 5525


In [1167]:
## tag the documents for doc2vec
class LabeledLineSentence(object):
    def __init__(self, texts):
        self.texts = texts
    def __iter__(self):
        for uid, line in enumerate(texts):
            yield LabeledSentence(words=line, tags=['SENT_%s' % uid])
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

In [1168]:
## Make a tagged corpora
sentences=LabeledLineSentence(texts+texts_whole+texts_pairs+texts_val+texts_whole_val+texts_pairs_val)

In [1169]:
# ## Build the vocabulary for the model 
model = Doc2Vec(min_count=5, window=7, size=100, sample=1e-4, negative=5, workers=8)
model.build_vocab(sentences)

In [1170]:
model.train(sentences, epochs=15, total_examples=model.corpus_count)

In [1171]:
l1=model.infer_vector("I work at a bank.")
l2=model.infer_vector("But I want to switch.")
cos_sim(l1,l2)  ## continuos sentences are well above 0.5 , looks good.

0.8701848

In [1172]:
model.similar_by_word('bank')

[('libor', 0.82145094871521),
 ('deposit', 0.8179093599319458),
 ('repo', 0.7785404920578003),
 ('paribas', 0.7619561553001404),
 ('swiss', 0.7556368112564087),
 ('funds', 0.7500659227371216),
 ('bnp', 0.7494849562644958),
 ('collateral', 0.7423702478408813),
 ('central', 0.7392298579216003),
 ('banks', 0.7346466779708862)]

In [1173]:
## Just a checkup for a random word

model.most_similar('good')  ## Not bad considering the very limited vocab

[('pretty', 0.9626896977424622),
 ('lot', 0.9598444700241089),
 ('really', 0.9581363201141357),
 ('looking', 0.9578802585601807),
 ('got', 0.9513303637504578),
 ('probably', 0.942491888999939),
 ('bit', 0.9405420422554016),
 ('better', 0.9350419044494629),
 ('going', 0.9350090026855469),
 ('little', 0.9320790767669678)]

In [1190]:
## good low similarity for different topics
cos_sim(model.infer_vector(texts[0]),model.infer_vector(texts[1]))  

0.2677874

In [1193]:
## good high similarity for sentences of same topics
cos_sim(model.infer_vector(texts_pairs[0]),model.infer_vector(texts_pairs[1]))  

0.8309205

In [66]:
## Build and test on Test documents

In [760]:
raw = []
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/test-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    raw = raw + [data]

In [69]:
## Extract acronymns that could make the sent_tokenizer split the sentence at them
pattern = r'(?:[A-Z]\.)+'
print re.findall(pattern,raw[0])

['U.S.', 'M.', 'E.N.', 'D.', 'G.']


In [70]:
abbreviatins=[re.findall(pattern, x) for x in raw]
abbreviatins=[item.lower() for sublist in abbreviatins for item in sublist]

In [71]:
## Give the Punkt tokenizer the extracted extra acronyms
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
punkt_param = PunktParameters()
punkt_param.abbrev_types = set(abbreviatins)
sentence_splitter = PunktSentenceTokenizer(punkt_param)
text = raw[0]
sentence_splitter.tokenize(text)

## The tokenizer cleary is very limited even after adding extra acronyms
## Will have to seperate sentences with regex with all the corner cases, not cool though!

['NEW YORK  (Reuters) - The number of U.S.',
 'workers filing new claims for jobless benefits fell more than expected last week, moderating from a 26-year high touched the previous week, Labor Department data showed on Thursday.KEY POINTS: * Despite the decline, jobless claims remain exceptionally high, and are more than 200,000 higher than a year ago.This pushed the Nasdaq up 4 percent for the week so far -- its best three-day percentage gain in a year.',
 "Financial companies and home builders, which have been pummeled in recent weeks as the housing market faltered, shone as investors took comfort in the Fed's view that the economy was likely to keep growing despite turmoil in credit markets.HOUSTON  (Reuters) - Stanford Financial Group's customer accounts are frozen until legal claims can be sorted out, and customers can't cash out their holdings at this time, the company's federally-appointed receiver said on Friday.",
 '"Customers may not transfer their accounts to another firm or

In [761]:
test_sentences1 = []
for i in raw:
    test_sentences1 = test_sentences1 + split_into_sentences(i)

In [762]:
## looks way better than Punkt tokenizer
test_sentences1

['NEW YORK  (Reuters) - The number of U.S. workers filing new claims for jobless benefits fell more than expected last week, moderating from a 26-year high touched the previous week, Labor Department data showed on Thursday.',
 'KEY POINTS: * Despite the decline, jobless claims remain exceptionally high, and are more than 200,000 higher than a year ago.',
 'This pushed the Nasdaq up 4 percent for the week so far -- its best three-day percentage gain in a year.',
 "Financial companies and home builders, which have been pummeled in recent weeks as the housing market faltered, shone as investors took comfort in the Fed's view that the economy was likely to keep growing despite turmoil in credit markets.",
 'HOUSTON  (Reuters) - Stanford Financial Group\'s customer accounts are frozen until legal claims can be sorted out, and customers can\'t cash out their holdings at this time, the company\'s federally-appointed receiver said on Friday".',
 'Customers may not transfer their accounts to a

In [763]:
## (0,1,2,3) *** (4,5) *** (6) *** (7,8) are the actual different topics
## and the model agrees too!
for c,i in enumerate(test_sentences1):
    if c!=len(test_sentences1)-1:
        sent1 = clean_string(test_sentences1[c],False)
        sent2 = clean_string(test_sentences1[c+1],False)
        sent1 = [word for word in sent1.lower().split() if word not in stoplist]
        sent2 = [word for word in sent2.lower().split() if word not in stoplist]
        
        print c,"->",c+1,":",cos_sim(model.infer_vector(sent1),model.infer_vector(sent2))


0 -> 1 : 0.80573976
1 -> 2 : 0.72347957
2 -> 3 : 0.8085551
3 -> 4 : 0.36068892
4 -> 5 : 0.5513739
5 -> 6 : -0.043770123
6 -> 7 : 0.15547763
7 -> 8 : 0.5196804
8 -> 9 : 0.5032281
9 -> 10 : 0.6318244
10 -> 11 : -0.4284422
11 -> 12 : -0.049857914
12 -> 13 : -0.3492122
13 -> 14 : 0.6622271
14 -> 15 : 0.4431028
15 -> 16 : 0.8595968
16 -> 17 : 0.63380784
17 -> 18 : 0.8910661
18 -> 19 : 0.80658853
19 -> 20 : 0.6607176
20 -> 21 : 0.17587599
21 -> 22 : 1.0
22 -> 23 : 0.05334391
23 -> 24 : 0.50937617
24 -> 25 : 0.43554688
25 -> 26 : 0.86094135
26 -> 27 : 0.45776945
27 -> 28 : 0.71657866
28 -> 29 : 0.87012696
29 -> 30 : 0.31589332
30 -> 31 : 0.4986119
31 -> 32 : 0.60362333
32 -> 33 : 0.48811927
33 -> 34 : -0.19401534
34 -> 35 : 0.2751177
35 -> 36 : 0.76278454
36 -> 37 : 0.6872997
37 -> 38 : 0.53975356
38 -> 39 : 0.5018099
39 -> 40 : 0.83363503
40 -> 41 : 0.5598384
41 -> 42 : 0.48769245
42 -> 43 : 0.098448925
43 -> 44 : 0.5799968
44 -> 45 : 0.12598225
45 -> 46 : -0.32094994
46 -> 47 : -0.34875044


404 -> 405 : 0.46128604
405 -> 406 : 0.36040363
406 -> 407 : 0.15956692
407 -> 408 : 0.40323627
408 -> 409 : 0.13700765
409 -> 410 : 0.24217866
410 -> 411 : 0.07936575
411 -> 412 : 0.8209041
412 -> 413 : 0.2439263
413 -> 414 : 0.12305562
414 -> 415 : -0.16567844
415 -> 416 : -0.24745348
416 -> 417 : -0.15916753
417 -> 418 : -0.41661698
418 -> 419 : 0.1191738
419 -> 420 : 0.29868314
420 -> 421 : 0.27668417
421 -> 422 : 0.53293455
422 -> 423 : 0.54607993
423 -> 424 : -0.030553728
424 -> 425 : 0.8173657
425 -> 426 : 0.4605935
426 -> 427 : 0.06250115
427 -> 428 : 0.3575245
428 -> 429 : 0.1075207
429 -> 430 : 0.602145
430 -> 431 : 0.3945033
431 -> 432 : 0.6228651
432 -> 433 : 0.3959409
433 -> 434 : 0.58491075
434 -> 435 : -0.20198385
435 -> 436 : 0.7323448
436 -> 437 : -0.08528444
437 -> 438 : 0.48114312
438 -> 439 : 0.21441482
439 -> 440 : 0.718503
440 -> 441 : 0.10706563
441 -> 442 : 0.91234595
442 -> 443 : 0.96663195
443 -> 444 : 0.8107899
444 -> 445 : 0.83337665
445 -> 446 : 0.3900667
4

803 -> 804 : 0.82021517
804 -> 805 : 0.6891477
805 -> 806 : 0.6275568
806 -> 807 : 0.36746445
807 -> 808 : 0.34225127
808 -> 809 : 0.08994599
809 -> 810 : 0.72795475
810 -> 811 : 0.31652704
811 -> 812 : 0.62032914
812 -> 813 : 0.8783668
813 -> 814 : 0.8517991
814 -> 815 : 0.38481545
815 -> 816 : 0.7875333
816 -> 817 : 0.69733936
817 -> 818 : 0.64184606
818 -> 819 : 0.5188541
819 -> 820 : 0.43098322
820 -> 821 : 0.92305934
821 -> 822 : 0.88018554
822 -> 823 : 0.39809376
823 -> 824 : 0.43526512
824 -> 825 : -0.12798215
825 -> 826 : -0.02458836
826 -> 827 : 0.770636
827 -> 828 : 0.57970005
828 -> 829 : 0.267916
829 -> 830 : 0.68126273
830 -> 831 : 0.18454398
831 -> 832 : 0.7484833
832 -> 833 : 0.5618892
833 -> 834 : 0.53187805
834 -> 835 : 0.6290227
835 -> 836 : 0.5924081
836 -> 837 : 0.40848207
837 -> 838 : 0.6874656
838 -> 839 : 0.1896626
839 -> 840 : 0.7292651
840 -> 841 : 0.04298602
841 -> 842 : 0.63712215
842 -> 843 : 0.8563351
843 -> 844 : 0.7670132
844 -> 845 : 0.750233
845 -> 846 

1171 -> 1172 : 0.033523753
1172 -> 1173 : 0.72748256
1173 -> 1174 : 0.80119133
1174 -> 1175 : 0.42134747
1175 -> 1176 : 0.731598
1176 -> 1177 : 0.1128277
1177 -> 1178 : 0.3685978
1178 -> 1179 : 0.73170334
1179 -> 1180 : 0.50146484
1180 -> 1181 : 0.67124724
1181 -> 1182 : 0.76258004
1182 -> 1183 : 0.6019955
1183 -> 1184 : 0.73498863
1184 -> 1185 : 0.68734366
1185 -> 1186 : 0.07953781
1186 -> 1187 : 0.120107405
1187 -> 1188 : 0.3628322
1188 -> 1189 : 0.6047237
1189 -> 1190 : -0.23870264
1190 -> 1191 : 0.20857793
1191 -> 1192 : 0.73749685
1192 -> 1193 : 0.8546829
1193 -> 1194 : 0.7086612
1194 -> 1195 : 0.35764903
1195 -> 1196 : 0.24320614
1196 -> 1197 : -0.117643885
1197 -> 1198 : 0.067896344
1198 -> 1199 : 0.6536711
1199 -> 1200 : 0.72357786
1200 -> 1201 : 0.4002724
1201 -> 1202 : 0.7068445
1202 -> 1203 : 0.18515429
1203 -> 1204 : 0.7833892
1204 -> 1205 : 0.4336153
1205 -> 1206 : 0.87455255
1206 -> 1207 : 0.47142074
1207 -> 1208 : 0.58946437
1208 -> 1209 : 0.30268073
1209 -> 1210 : 0.176

1521 -> 1522 : 0.23341708
1522 -> 1523 : 0.72713816
1523 -> 1524 : 0.8377976
1524 -> 1525 : -0.033234697
1525 -> 1526 : 0.5236533
1526 -> 1527 : 0.4799185
1527 -> 1528 : 0.112247825
1528 -> 1529 : 0.49718076
1529 -> 1530 : 0.6432067
1530 -> 1531 : 0.83601314
1531 -> 1532 : 0.24054846
1532 -> 1533 : 0.81428003
1533 -> 1534 : 0.7026223
1534 -> 1535 : 0.5821025
1535 -> 1536 : 0.4524212
1536 -> 1537 : 0.75044316
1537 -> 1538 : -0.017371798
1538 -> 1539 : 0.19548233
1539 -> 1540 : 0.33146963
1540 -> 1541 : -0.3263187
1541 -> 1542 : 0.07147689
1542 -> 1543 : 0.62721336
1543 -> 1544 : 0.64893204
1544 -> 1545 : 0.77201945
1545 -> 1546 : 0.4661438
1546 -> 1547 : 0.4632835
1547 -> 1548 : -0.3062156
1548 -> 1549 : 0.49352425
1549 -> 1550 : 0.67385846
1550 -> 1551 : 0.49962637
1551 -> 1552 : 0.16258408
1552 -> 1553 : 0.61987984
1553 -> 1554 : 0.7203899
1554 -> 1555 : 0.67074406
1555 -> 1556 : 0.20695558
1556 -> 1557 : 0.8197379
1557 -> 1558 : 0.5102984
1558 -> 1559 : 0.8662801
1559 -> 1560 : 0.336

...

2170 -> 2171 : 0.59980625
2171 -> 2172 : -0.00281664
2172 -> 2173 : 0.627585
2173 -> 2174 : 0.5514014
2174 -> 2175 : -0.03500738
2175 -> 2176 : 0.6922311
2176 -> 2177 : 0.65976834
2177 -> 2178 : 0.5137715
2178 -> 2179 : 0.5245696
2179 -> 2180 : 0.7669648
2180 -> 2181 : 0.49503055
2181 -> 2182 : 0.5670265
2182 -> 2183 : 0.6048587
2183 -> 2184 : 0.35848603
2184 -> 2185 : 0.18110737
2185 -> 2186 : 0.79581696
2186 -> 2187 : 0.78112674
2187 -> 2188 : 0.2428041
2188 -> 2189 : 0.6825113
2189 -> 2190 : 0.12293764
2190 -> 2191 : 0.0056767403
2191 -> 2192 : 0.04636425
2192 -> 2193 : 1.0
2193 -> 2194 : -0.0030029092
2194 -> 2195 : 0.6086795
2195 -> 2196 : 0.73662126
2196 -> 2197 : 0.62189925
2197 -> 2198 : 0.44142738
2198 -> 2199 : 0.49893627
2199 -> 2200 : 0.42127407
2200 -> 2201 : 0.3423755
2201 -> 2202 : 0.81266737
2202 -> 2203 : 0.7986211
2203 -> 2204 : 0.24694028
2204 -> 2205 : 0.6119525
2205 -> 2206 : 0.58975756
2206 -> 2207 : 0.7314715
2207 -> 2208 : 0.86281425
2208 -> 2209 : 0.72747666
22

2489 -> 2490 : 0.19269979
2490 -> 2491 : 0.83166265
2491 -> 2492 : 0.5500815
2492 -> 2493 : 0.67124295
2493 -> 2494 : 0.65993327
2494 -> 2495 : 0.317441
2495 -> 2496 : 0.58126634
2496 -> 2497 : 0.74008584
2497 -> 2498 : 0.41614154
2498 -> 2499 : 0.6073681
2499 -> 2500 : 0.6785592
2500 -> 2501 : 0.60159624
2501 -> 2502 : 0.39892066
2502 -> 2503 : 0.20913568
2503 -> 2504 : 0.67070425
2504 -> 2505 : 0.79901904
2505 -> 2506 : 0.040312406
2506 -> 2507 : 0.3880415
2507 -> 2508 : 0.35439894
2508 -> 2509 : 0.80237377
2509 -> 2510 : 0.81974334
2510 -> 2511 : 0.37957728
2511 -> 2512 : 0.82758963
2512 -> 2513 : 0.18552499
2513 -> 2514 : 0.30991453
2514 -> 2515 : 0.52238244
2515 -> 2516 : 0.29904836
2516 -> 2517 : -0.09544673
2517 -> 2518 : 0.61549926
2518 -> 2519 : -0.46430147
2519 -> 2520 : -0.05348486
2520 -> 2521 : 0.6849042
2521 -> 2522 : 0.17948093
2522 -> 2523 : 0.16256303
2523 -> 2524 : 0.7021119
2524 -> 2525 : 0.8440167
2525 -> 2526 : 0.7026025
2526 -> 2527 : 0.67574286
2527 -> 2528 : 0.6

2817 -> 2818 : -0.3377151
2818 -> 2819 : 0.60660076
2819 -> 2820 : 0.7463082
2820 -> 2821 : -0.047315728
2821 -> 2822 : 0.82067597
2822 -> 2823 : 0.613681
2823 -> 2824 : 0.5714053
2824 -> 2825 : 0.7967078
2825 -> 2826 : 0.7508381
2826 -> 2827 : 0.4959166
2827 -> 2828 : 0.5073702
2828 -> 2829 : 0.31582823
2829 -> 2830 : -0.24250658
2830 -> 2831 : 0.7715696
2831 -> 2832 : 0.51992065
2832 -> 2833 : 0.46509153
2833 -> 2834 : 0.012501734
2834 -> 2835 : 0.50867933
2835 -> 2836 : 0.055919204
2836 -> 2837 : -0.06769847
2837 -> 2838 : 0.48604107
2838 -> 2839 : 0.049796157
2839 -> 2840 : 0.663763
2840 -> 2841 : 0.63540065
2841 -> 2842 : 0.6560989
2842 -> 2843 : 0.023331426
2843 -> 2844 : -0.031846978
2844 -> 2845 : -0.014760936
2845 -> 2846 : -0.008734706
2846 -> 2847 : 0.8706305
2847 -> 2848 : 0.7800251
2848 -> 2849 : 0.56577945
2849 -> 2850 : 0.53448534
2850 -> 2851 : 0.54228044
2851 -> 2852 : 0.33808088
2852 -> 2853 : 0.7946132
2853 -> 2854 : 0.89395636
2854 -> 2855 : 0.55696714
2855 -> 2856 

3139 -> 3140 : 0.6974518
3140 -> 3141 : 0.9003654
3141 -> 3142 : 0.21382688
3142 -> 3143 : 0.8878277
3143 -> 3144 : 0.84187925
3144 -> 3145 : 0.41215855
3145 -> 3146 : 0.6231901
3146 -> 3147 : 0.553295
3147 -> 3148 : -0.0049336157
3148 -> 3149 : 0.009991641
3149 -> 3150 : -0.39107233
3150 -> 3151 : 0.7088908
3151 -> 3152 : 0.4719136
3152 -> 3153 : 0.7638014
3153 -> 3154 : 0.48964933
3154 -> 3155 : 0.74787724
3155 -> 3156 : 0.91236544
3156 -> 3157 : 0.82362425
3157 -> 3158 : 0.86905396
3158 -> 3159 : 0.8170413
3159 -> 3160 : 0.65360576
3160 -> 3161 : 0.58402777
3161 -> 3162 : 0.5312339
3162 -> 3163 : 0.6468385
3163 -> 3164 : 0.8027152
3164 -> 3165 : 0.8571134
3165 -> 3166 : 0.73348063
3166 -> 3167 : 0.5518636
3167 -> 3168 : 0.22776389
3168 -> 3169 : 0.42191815
3169 -> 3170 : 0.38715965
3170 -> 3171 : 0.44675103
3171 -> 3172 : -0.31261262
3172 -> 3173 : 0.6945648
3173 -> 3174 : 0.74754214
3174 -> 3175 : 0.7049597
3175 -> 3176 : 0.7652802
3176 -> 3177 : -0.22282535
3177 -> 3178 : 0.218044

3465 -> 3466 : 0.74716157
3466 -> 3467 : -0.010912534
3467 -> 3468 : 0.5810413
3468 -> 3469 : 0.40000933
3469 -> 3470 : 0.21538277
3470 -> 3471 : 0.33742595
3471 -> 3472 : 0.17468959
3472 -> 3473 : 0.7241496
3473 -> 3474 : 0.2735043
3474 -> 3475 : -0.26750138
3475 -> 3476 : 0.7284134
3476 -> 3477 : 0.08918704
3477 -> 3478 : 0.8223122
3478 -> 3479 : 0.69252795
3479 -> 3480 : 0.5457387
3480 -> 3481 : 0.5965747
3481 -> 3482 : 0.31722745
3482 -> 3483 : 0.47311836
3483 -> 3484 : 0.24066703
3484 -> 3485 : 0.6045064
3485 -> 3486 : 0.6435119
3486 -> 3487 : 0.72986704
3487 -> 3488 : 0.52957857
3488 -> 3489 : 0.71789944
3489 -> 3490 : 0.43558833
3490 -> 3491 : 0.7073183
3491 -> 3492 : 0.72765887
3492 -> 3493 : 0.69190866
3493 -> 3494 : 0.4558427
3494 -> 3495 : 0.5589505
3495 -> 3496 : 0.8102448
3496 -> 3497 : -0.06479429
3497 -> 3498 : 0.41740298
3498 -> 3499 : 0.39226604
3499 -> 3500 : 0.76382875
3500 -> 3501 : 0.8433185
3501 -> 3502 : 0.48334375
3502 -> 3503 : 0.23492998
3503 -> 3504 : 0.65824

In [630]:
test_sentences1

['NEW YORK  (Reuters) - The number of U.S. workers filing new claims for jobless benefits fell more than expected last week, moderating from a 26-year high touched the previous week, Labor Department data showed on Thursday.',
 'KEY POINTS: * Despite the decline, jobless claims remain exceptionally high, and are more than 200,000 higher than a year ago.',
 'This pushed the Nasdaq up 4 percent for the week so far -- its best three-day percentage gain in a year.',
 "Financial companies and home builders, which have been pummeled in recent weeks as the housing market faltered, shone as investors took comfort in the Fed's view that the economy was likely to keep growing despite turmoil in credit markets.",
 'HOUSTON  (Reuters) - Stanford Financial Group\'s customer accounts are frozen until legal claims can be sorted out, and customers can\'t cash out their holdings at this time, the company\'s federally-appointed receiver said on Friday".',
 'Customers may not transfer their accounts to a

In [764]:
## load text files
test_files = []
file_names = []
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/test-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    test_files.append([data])
    file_names.append(filename)

In [1194]:
## dict for text files with
  ## keys as file names
  ## values as txt as a list
test_dictionary = {}
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/test-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    test_dictionary[filename] = data


In [1195]:
## create a dictionary with 
    #  keys as filenames
    #  values as list of indices where each sentences end in the filename
sentences_ending_indices = defaultdict(list)

for c,one_file in enumerate(test_files):
    file_sentences = split_into_sentences(one_file[0])
    temp = []
    start_ind = 0
    temp.append(start_ind)
    for sent in file_sentences:
        end_ind = start_ind + len(sent)
        temp.append(end_ind)
        start_ind = end_ind
    sentences_ending_indices[file_names[c]] = temp
    
print sentences_ending_indices['/Users/shashankjakka/Downloads/amex2/test-data/test_131.txt']

[0, 76, 213, 314, 422, 494, 591, 823, 923, 1073, 1113, 1308, 1399, 1551, 1852, 1956, 2054, 2189]


In [1196]:
## correct indices! 
split_into_sentences(list(compress(test_files,np.array(file_names) == 
                                   '/Users/shashankjakka/Downloads/amex2/test-data/test_131.txt'))[0][0])

['With gasoline prices rising, inflation pressures picked up a bit last month.',
 'A price index for personal consumer expenditures increased 0.4 percent, the largest gain since March last year, after being flat in July.',
 'In the 12 months through August, the PCE index rose 1.5 percent after increasing 1.3 percent in July.',
 "But their parents will be spending less on their children's apparel, shoes and accessories, the survey said.",
 'Parents said annual spending on teens is down 19 percent from last year.',
 'But mom and dad are spending even less on themselves, with a 26 percent decline, the survey said.',
 'Many apparel retailers, such as Children\'s Place Retail Stores Inc ( PLCE.O. ) and AnnTaylor Stores Corp ( ANN.N. ) have been hit with weak sales as high gas prices and the widening housing downturn have kept many shoppers at home".',
 'Capital spending has turned positive, household consumption is strong and exports continued to rise.',
 'Japan\'s economy is doing fairly w

In [1197]:
## Now split each text file into sentences
test_sentences = []
for each_file in test_files:
    test_sentences.append(split_into_sentences(each_file[0]))

In [1198]:
len(test_sentences)

250

In [770]:
## Get the boundaries for each text file as a dictionary
## the model infered vector will be different each time ( non- deterministic)
breaks = defaultdict(list)
for c,each_file_sentences in enumerate(test_sentences):
    filename = file_names[c]
    indices = sentences_ending_indices[filename]
    for count,each_sentence in enumerate(each_file_sentences):
        if count != len(each_file_sentences)-1:
            sent1 = clean_string(each_file_sentences[count], False)
            sent2 = clean_string(each_file_sentences[count + 1], False)
            sent1 = [word for word in sent1.lower().split() if word not in stoplist]
            sent2 = [word for word in sent2.lower().split() if word not in stoplist]
            threshould = (cos_sim(model.infer_vector(sent1),model.infer_vector(sent2)))
            print count,'->',count+1,':',threshould 
            if abs(threshould) < 0.4:
                breaks[filename].append(indices[count + 1])

0 -> 1 : 0.80778956
1 -> 2 : 0.54894155
2 -> 3 : 0.7858891
3 -> 4 : 0.20911029
4 -> 5 : 0.413889
5 -> 6 : -0.059838448
6 -> 7 : 0.3646527
7 -> 8 : 0.68262184
0 -> 1 : 0.83879554
1 -> 2 : -0.2838935
2 -> 3 : 0.020727502
3 -> 4 : -0.28419086
4 -> 5 : 0.7517663
5 -> 6 : 0.47155273
6 -> 7 : 0.85678655
7 -> 8 : 0.5841972
8 -> 9 : 0.90131813
9 -> 10 : 0.79187244
0 -> 1 : 0.16196465
1 -> 2 : 1.0
2 -> 3 : 0.08522239
3 -> 4 : 0.3089258
4 -> 5 : 0.24967661
5 -> 6 : 0.46100834
6 -> 7 : 0.43067396
7 -> 8 : 0.82305175
8 -> 9 : 0.82762516
9 -> 10 : 0.19190094
10 -> 11 : 0.17182812
11 -> 12 : 0.68083084
12 -> 13 : 0.5869451
13 -> 14 : -0.316456
14 -> 15 : -0.014840594
0 -> 1 : 0.7499335
1 -> 2 : 0.15591927
2 -> 3 : 0.47209787
3 -> 4 : 0.69226426
4 -> 5 : 0.3328486
5 -> 6 : 0.38731423
6 -> 7 : -0.24291624
7 -> 8 : 0.69851464
8 -> 9 : 0.117232025
9 -> 10 : -0.34226757
10 -> 11 : -0.44842765
0 -> 1 : 0.055590242
1 -> 2 : 0.30239138
2 -> 3 : 0.1526258
3 -> 4 : 0.6408967
4 -> 5 : 0.87347054
5 -> 6 : 0.444

8 -> 9 : 0.55594885
9 -> 10 : 0.63249266
10 -> 11 : 0.48908222
0 -> 1 : 0.58592266
1 -> 2 : 0.6367537
2 -> 3 : 0.5520568
3 -> 4 : 0.3061509
4 -> 5 : 0.39051434
5 -> 6 : -0.30493292
6 -> 7 : 0.5828396
7 -> 8 : 0.8357722
8 -> 9 : -0.14397834
9 -> 10 : 0.898276
10 -> 11 : 0.38939822
11 -> 12 : 0.9667311
12 -> 13 : 0.7844674
0 -> 1 : 0.4648217
1 -> 2 : 0.43832117
2 -> 3 : 0.027052348
3 -> 4 : 1.0
4 -> 5 : 0.07101289
5 -> 6 : 0.44950303
6 -> 7 : 0.59765404
7 -> 8 : 0.5608144
8 -> 9 : 0.47459832
9 -> 10 : 0.4337757
10 -> 11 : 0.27341604
11 -> 12 : 0.6065156
12 -> 13 : 0.29810885
13 -> 14 : 0.41069555
14 -> 15 : 0.35553402
15 -> 16 : -0.16856052
0 -> 1 : 0.89864445
1 -> 2 : 0.84387136
2 -> 3 : 0.8139846
3 -> 4 : -0.19876045
4 -> 5 : 0.792645
5 -> 6 : 0.6240486
6 -> 7 : 0.25873187
7 -> 8 : 0.5264712
8 -> 9 : 0.42222154
9 -> 10 : 0.43172058
10 -> 11 : 0.76304096
11 -> 12 : 0.93198717
12 -> 13 : 0.32673278
0 -> 1 : 0.5610145
1 -> 2 : 0.5137246
2 -> 3 : 0.01772792
3 -> 4 : 0.92918646
4 -> 5 : 0.2

3 -> 4 : 0.81108147
4 -> 5 : 0.36006865
5 -> 6 : 0.27258965
6 -> 7 : 0.36097896
7 -> 8 : 0.27695647
8 -> 9 : 0.8411226
9 -> 10 : 0.27835482
10 -> 11 : 0.14817491
0 -> 1 : -0.040568322
1 -> 2 : 0.50410134
2 -> 3 : 0.23455518
3 -> 4 : -0.20759925
4 -> 5 : 1.0
5 -> 6 : 0.061044406
6 -> 7 : 0.8815589
7 -> 8 : 0.6074973
8 -> 9 : 0.7414685
9 -> 10 : 0.8147752
10 -> 11 : -0.05631632
11 -> 12 : 0.64722526
12 -> 13 : 0.81147116
13 -> 14 : 0.39754972
14 -> 15 : 0.28626165
15 -> 16 : 0.6393064
16 -> 17 : 0.48623586
17 -> 18 : 0.45601112
18 -> 19 : 0.22683279
19 -> 20 : 0.69423056
20 -> 21 : 0.0018480241
0 -> 1 : 0.2897473
1 -> 2 : 0.3548081
2 -> 3 : 0.048874665
3 -> 4 : 0.64483076
4 -> 5 : -0.05143068
5 -> 6 : 0.31834674
6 -> 7 : 0.21929094
7 -> 8 : 0.50927764
8 -> 9 : 0.5834261
9 -> 10 : 0.33935654
10 -> 11 : 0.77030617
0 -> 1 : 0.824127
1 -> 2 : -0.21617813
2 -> 3 : -0.20009516
3 -> 4 : 0.3843975
4 -> 5 : 0.32664734
5 -> 6 : 0.69207734
6 -> 7 : 0.74903095
7 -> 8 : 0.8076038
8 -> 9 : 0.51798713


16 -> 17 : 0.0972106
17 -> 18 : 0.4217253
18 -> 19 : 0.47376198
19 -> 20 : 0.62858826
20 -> 21 : 0.4121833
21 -> 22 : 0.60830426
0 -> 1 : 0.8362344
1 -> 2 : 0.80449456
2 -> 3 : -0.0058997283
3 -> 4 : 0.60267264
4 -> 5 : 0.50945634
5 -> 6 : 0.83558124
6 -> 7 : 0.8380548
7 -> 8 : 0.8646555
8 -> 9 : 0.7359051
9 -> 10 : 0.7860903
10 -> 11 : 0.6233962
11 -> 12 : 0.90273196
12 -> 13 : 0.7601297
0 -> 1 : 0.5495193
1 -> 2 : 0.6971323
2 -> 3 : 0.81599385
3 -> 4 : -0.19970968
4 -> 5 : 0.055481076
5 -> 6 : 0.54747504
6 -> 7 : 0.62736773
7 -> 8 : 0.7255832
8 -> 9 : -0.38804978
9 -> 10 : 0.3193851
0 -> 1 : 0.5539982
1 -> 2 : 0.69508106
2 -> 3 : 0.8176247
3 -> 4 : 0.71819025
4 -> 5 : 0.69437104
5 -> 6 : 0.83857447
6 -> 7 : 0.19725677
7 -> 8 : 0.61483204
8 -> 9 : 0.5842611
9 -> 10 : 0.7030293
0 -> 1 : 0.73090005
1 -> 2 : 0.6573147
2 -> 3 : 0.005818083
3 -> 4 : -0.5563347
4 -> 5 : 0.80416816
5 -> 6 : 0.22622159
6 -> 7 : 0.4052899
7 -> 8 : 0.6405556
8 -> 9 : 0.09689878
9 -> 10 : 0.59912753
10 -> 11 : -

6 -> 7 : 0.79626596
7 -> 8 : 0.26589462
8 -> 9 : -0.0002905795
9 -> 10 : 1.0
10 -> 11 : 0.01428381
11 -> 12 : 0.63717717
12 -> 13 : 0.16591832
13 -> 14 : 0.3798886
14 -> 15 : 0.26838923
0 -> 1 : 0.1157197
1 -> 2 : 0.6347135
2 -> 3 : 0.5537844
3 -> 4 : 0.59389704
4 -> 5 : 0.28191376
5 -> 6 : 0.5068595
6 -> 7 : 0.59453523
7 -> 8 : 0.16948143
8 -> 9 : 0.41484284
9 -> 10 : -0.38439578
10 -> 11 : 0.53266346
0 -> 1 : 0.52122355
1 -> 2 : 0.72010154
2 -> 3 : 0.37574667
3 -> 4 : 0.06424794
4 -> 5 : -0.6697595
5 -> 6 : 0.6376666
6 -> 7 : 0.62112755
7 -> 8 : 0.35690114
8 -> 9 : 0.11154563
9 -> 10 : 0.59561884
10 -> 11 : 0.30204588
11 -> 12 : 0.6810781
12 -> 13 : 0.7122038
0 -> 1 : 0.454749
1 -> 2 : 0.6452161
2 -> 3 : 0.47220865
3 -> 4 : 0.009969216
4 -> 5 : 0.04795184
5 -> 6 : 0.19038007
6 -> 7 : -0.04531064
7 -> 8 : 0.26012182
8 -> 9 : -0.02759016
9 -> 10 : -0.07493022
10 -> 11 : 0.50196445
11 -> 12 : 0.31326112
0 -> 1 : 0.69646657
1 -> 2 : 0.1426239
2 -> 3 : 0.44091043
3 -> 4 : 0.77475387
4 -> 

3 -> 4 : -0.13885552
4 -> 5 : 0.60669327
5 -> 6 : 0.79639393
6 -> 7 : 0.58978564
7 -> 8 : 0.39954293
8 -> 9 : 0.9664273
9 -> 10 : 0.46736097
10 -> 11 : 0.90447545
11 -> 12 : 0.099523865
12 -> 13 : 0.6662041
13 -> 14 : 0.3125296
14 -> 15 : 0.15573625
15 -> 16 : 0.45161945
0 -> 1 : 0.3786421
1 -> 2 : -0.26969865
2 -> 3 : 0.7245699
3 -> 4 : -0.035943523
4 -> 5 : 0.38807517
5 -> 6 : 0.3389017
6 -> 7 : 0.7529772
7 -> 8 : 0.31489444
8 -> 9 : 0.11352911
9 -> 10 : 0.88353443
10 -> 11 : 0.34155324
11 -> 12 : 0.3460711
0 -> 1 : 0.8213798
1 -> 2 : 0.8178149
2 -> 3 : 0.19353221
3 -> 4 : 0.906711
4 -> 5 : 0.49637672
5 -> 6 : 0.7601206
6 -> 7 : 0.4738811
7 -> 8 : 0.7501942
8 -> 9 : 0.5517208
9 -> 10 : 0.85293466
10 -> 11 : 0.7153018
11 -> 12 : 0.24917172
12 -> 13 : -0.16071531
13 -> 14 : 0.30675387
14 -> 15 : 0.0712789
0 -> 1 : 0.12589878
1 -> 2 : -0.25534862
2 -> 3 : 0.6879734
3 -> 4 : 0.11874212
4 -> 5 : 0.55397373
5 -> 6 : 0.5152378
6 -> 7 : 0.42173567
7 -> 8 : 0.50176764
8 -> 9 : 0.767448
9 -> 1

1 -> 2 : 0.45111597
2 -> 3 : 0.7057416
3 -> 4 : 0.8750517
4 -> 5 : 0.79721797
5 -> 6 : 0.28250968
6 -> 7 : 0.54886585
7 -> 8 : 0.5089071
8 -> 9 : -0.12914558
9 -> 10 : 0.7162725
10 -> 11 : 0.68241036
11 -> 12 : 0.4385389
12 -> 13 : 0.45235375
13 -> 14 : 0.6119733
14 -> 15 : 0.052973486
0 -> 1 : 0.027118623
1 -> 2 : 0.22874476
2 -> 3 : 0.70825875
3 -> 4 : 0.7653081
4 -> 5 : 0.51564497
5 -> 6 : 0.08039005
6 -> 7 : -0.04102408
7 -> 8 : -0.017428437
8 -> 9 : 0.23081878
9 -> 10 : 0.68670344
10 -> 11 : 0.71129835
11 -> 12 : 0.5083987
12 -> 13 : 0.5780725
13 -> 14 : 0.64864916
0 -> 1 : 0.79596704
1 -> 2 : 0.9338062
2 -> 3 : 0.6388761
3 -> 4 : 0.38465533
4 -> 5 : 0.88225114
5 -> 6 : 0.09506103
6 -> 7 : 0.10588265
7 -> 8 : 0.86423
8 -> 9 : 0.70501924
9 -> 10 : 0.25861982
10 -> 11 : 0.42500916
11 -> 12 : 0.2063389
12 -> 13 : 0.81386054
13 -> 14 : 0.40346372
14 -> 15 : 0.004692137
15 -> 16 : 0.6753808
0 -> 1 : 0.56878495
1 -> 2 : 0.73795754
2 -> 3 : -0.20154035
3 -> 4 : 0.74560827
4 -> 5 : -0.222

2 -> 3 : -0.059804123
3 -> 4 : 0.82429993
4 -> 5 : 0.27399558
5 -> 6 : 0.01992141
6 -> 7 : 0.602415
7 -> 8 : 0.6899958
8 -> 9 : 0.558006
9 -> 10 : 0.63591766
0 -> 1 : 0.8774578
1 -> 2 : 0.55736995
2 -> 3 : 0.25881365
3 -> 4 : 0.34706086
4 -> 5 : 0.33792758
5 -> 6 : 0.2512053
6 -> 7 : 0.013918146
7 -> 8 : 0.5263377
8 -> 9 : 0.2242493
9 -> 10 : 0.2646433
10 -> 11 : 0.66168016
11 -> 12 : 0.6828519
0 -> 1 : 0.440717
1 -> 2 : 0.7023202
2 -> 3 : 0.25309575
3 -> 4 : 0.52725524
4 -> 5 : -0.4099604
5 -> 6 : -0.15526167
6 -> 7 : 0.58898705
7 -> 8 : 0.40654525
8 -> 9 : 0.8284353
9 -> 10 : 0.42012632
10 -> 11 : 0.6907408
0 -> 1 : 0.67479396
1 -> 2 : 0.5388525
2 -> 3 : 0.71526515
3 -> 4 : -0.02646412
4 -> 5 : 0.8424967
5 -> 6 : 0.21436031
6 -> 7 : 0.7642949
7 -> 8 : 0.7073038
8 -> 9 : 0.41414198
9 -> 10 : 0.44428802
10 -> 11 : 0.33998474
11 -> 12 : 0.4119001
0 -> 1 : 0.45952076
1 -> 2 : 0.8724439
2 -> 3 : 0.8940338
3 -> 4 : 0.54949266
4 -> 5 : 0.22700033
5 -> 6 : 0.6862717
6 -> 7 : 0.016659608
7 ->

In [776]:
## make a dataset
formatted_breaks = {}
for key in breaks.keys():
    temp = breaks[key]
    temp = [str(x) for x in temp]
    temp = '-'.join(temp)
    formatted_breaks[key] = temp

In [777]:
## make a dataset

submission = pd.DataFrame.from_dict(formatted_breaks,orient='index').reset_index()
submission = submission.rename(columns={
                'index': 'file_name',
                     0 : 'segments'
            })
# submission = submission[~(submission.segments =='')]## this removed additionally 5 files, but the csv was accepted anyway!
submission.file_name = submission.file_name.apply(lambda x: x[47:])
submission = submission.sort_values('file_name').reset_index(drop = True)

In [778]:
## didnt add 0 and last sentences inds ,psst
submission.head()

,file_name,segments
0,test_1.txt,375-827-953-1209
1,test_10.txt,348-599-775-834-839-936-1042-1296-1557-1569-1807
2,test_100.txt,236-646-1175-1319-1473-1860-2587
3,test_101.txt,287-469
4,test_102.txt,826-933-1098-1768


In [779]:
## add initial 1
## add remaining last boundary which was not added previously
submission.segments=submission.segments.apply(lambda x: '1-'+x)
temp=list(submission.segments.values)[:]
c=0
for i,j in zip(submission.file_name,submission.segments.values):
    x=int(sentences_ending_indices['/Users/shashankjakka/Downloads/amex2/test-data/%s'%i][-1])+1
    temp[c]=j+'-'+str(x)
    c=c+1
submission.segments=temp

In [780]:
submission.head()

,file_name,segments
0,test_1.txt,1-375-827-953-1209-2008
1,test_10.txt,1-348-599-775-834-839-936-1042-1296-1557-1569-...
2,test_100.txt,1-236-646-1175-1319-1473-1860-2587-2890
3,test_101.txt,1-287-469-2151
4,test_102.txt,1-826-933-1098-1768-2212


In [782]:
submission.to_csv('/Users/shashankjakka/Downloads/amex2/fresh1minc7win15eps.csv',index=False)

In [ ]:
# make dataset for xgboost

In [836]:
vectors = []
for c,each_file_sentences in enumerate(test_sentences):
    filename = file_names[c]
#     indices = sentences_ending_indices[filename]
    for count,each_sentence in enumerate(each_file_sentences):
        if count != len(each_file_sentences)-1:
            sent1 = clean_string(each_file_sentences[count], False)
            sent1 = [word for word in sent1.lower().split() if word not in stoplist]
            vec = model.infer_vector(sent1)
            vectors.append([filename + '_' + str(count),list(vec)])
            
df=pd.DataFrame(vectors,columns=['file_name','vector'])
df.file_name = df.file_name.apply(lambda x: x[47:])

In [856]:
df = pd.concat([df,pd.DataFrame(df.vector.values.tolist(), index= df.index)],axis=1).drop('vector',axis=1)

In [868]:
X = df.iloc[:,1:]

In [945]:
# Dont re run the inferences will be diff every time
temp=[]
for filename in glob.glob('/Users/shashankjakka/Downloads/amex2/training-data/*'):
    fp = open(filename,'r')
    data = fp.read()
    data = data.split('\n\n-----\n\n')
    consequitivedots = re.compile(r'\.{3,}')
    data = [consequitivedots.sub('', x) for x in data] ## remove consecutive '.'
    data = [re.sub('\d+(?:\.\d+)?', '', x) for x in data]
    data = [re.sub(r'\([^\(]*?\)', r'', x) for x in data]
    temp = temp + data

with_boundaries = []
for i in (temp):
    with_boundaries = with_boundaries + split_into_sentences(i)
    with_boundaries.append(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    
for c,i in enumerate(with_boundaries):
    if i != ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>":
        with_boundaries[c] = clean_string(with_boundaries[c],False)
        with_boundaries[c] = [word for word in with_boundaries[c].lower().split() if word not in stoplist]
        with_boundaries[c] = model.infer_vector(with_boundaries[c])

In [951]:
train_vectors = [list(x) if x != '>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>' else x 
                    for x in with_boundaries]

In [1082]:
pair_vectors = []
labels = []
for c, i in enumerate(train_vectors):
    if c != len(train_vectors)-2:
        if i != '>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>':
            if train_vectors[c+1] != '>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>':
                pair_vectors.append(map(lambda x,y:x+y,train_vectors[c], train_vectors[c+1]))
                labels.append(0)
#                 print 'pairing %d and %d'%(c,c+1) + ' with label %d' %0 + ' difference %d'%(1)
                d = 1
            else:
                j = c+1
                while(train_vectors[j] == '>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>'):
                    j = j+1
                pair_vectors.append(map(lambda x,y: x+y,train_vectors[c],train_vectors[j])) ## no edge case, can do
                labels.append(1)
                d = (j-c)
#                 print 'pairing %d and %d'%(c,j) + ' with label %d' %1 + ' difference %d'%(j-c)
#     if d==3: print c,i   

In [1087]:
len(train_vectors)

42355

In [995]:
c=0
for i in train_vectors:
    if i=='>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>':
        c=c+1

In [997]:
len(pair_vectors)+c ## looks good

42354

In [1088]:
train = pd.DataFrame(pair_vectors)
train['labels'] = labels

In [1089]:
train.shape

(31303, 101)

In [1090]:
train['labels'].value_counts()

0    20290
1    11013
Name: labels, dtype: int64

In [1096]:
## gradient boosting
def run(train,lables,test,params):
    dtrain=xgb.DMatrix(train,label=lables)
    dtest=xgb.DMatrix(test)
    bst=xgb.train(dtrain=dtrain,params=params,num_boost_round=150)
    preds=bst.predict(dtest)
    return bst,preds

params={
    'objective':'binary:logistic',
    'max_depth':3,
#     'eta':0.3,
    'subsample':0.9,
#     'colsample_bytree':0.4
}
    

In [1097]:
X_train,X_test,y_train,y_test = train_test_split(train.iloc[:,:-1],train.iloc[:,-1],test_size=0.3)

In [1099]:
# bst,probs = run(X_train,y_train,X_test,params)
# preds = (probs>=0.5).astype('int')
# print accuracy_score(y_test,preds)
# print confusion_matrix(y_test,preds)

In [1066]:
dtrain = xgb.DMatrix(train.iloc[:,:-1].values,label=train.iloc[:,-1].values)
bst = xgb.train(dtrain=dtrain,params=params,num_boost_round=100)

In [1069]:
train.iloc[:,:-1].values.shape

(31303, 200)

In [1070]:
v=vec.reshape(-1,1)
v.shape

(100, 1)

In [1101]:
# make predictions
breaks = defaultdict(list)
for c,each_file_sentences in enumerate(test_sentences):
    filename = file_names[c]
    indices = sentences_ending_indices[filename]
    for count,each_sentence in enumerate(each_file_sentences):
        if count != len(each_file_sentences)-1:
            sent1 = clean_string(each_file_sentences[count], False)
            sent2 = clean_string(each_file_sentences[count + 1], False)
            sent1 = [word for word in sent1.lower().split() if word not in stoplist]
            sent2 = [word for word in sent2.lower().split() if word not in stoplist]
            vec1  = model.infer_vector(sent1)
            vec2  = model.infer_vector(sent2)
            vec = vec1 + vec2
            dtest=xgb.DMatrix(vec.reshape(-1,1))
            pred  = bst.predict(dtest)[0]
            if abs(threshould) < 0.5:
                breaks[filename].append(indices[count + 1])

In [1273]:
## final
sub

,file_name,segments
0,test_1.txt,0-375-827-1209-2008
1,test_10.txt,0-348-775-1296-2134
2,test_100.txt,0-646-1860-2889
3,test_101.txt,0-140-287-469-1468-2151
4,test_102.txt,0-476-826-933-1098-1768-2212
5,test_103.txt,0-1198-1597-1988
6,test_104.txt,0-323-427-606-747-1084-1496-1698
7,test_105.txt,0-114-224-1165-1301-1542-1982
8,test_106.txt,0-425-624-739-978-1744-2720
9,test_107.txt,0-381-1287-1914
